In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.openai_functions import OpenAIFunctionsChain
from langchain.prompts import ChatPromptTemplate

In [179]:
llm = ChatOpenAI(temperature=0, 
                   model="gpt-3.5-turbo-0613")

In [218]:
def create_extraction_chain(schema, llm, prompt):
    chain = OpenAIFunctionsChain(llm=llm, prompt=prompt, entity_schema=schema, tagging=False)
    return chain

def create_tagging_chain(schema, llm, prompt):
    chain = OpenAIFunctionsChain(llm=llm, prompt=prompt, entity_schema=schema, tagging=True)
    return chain

In [219]:
prompt = ChatPromptTemplate.from_template("""
                                            Extract and save the relevant entities mentioned in the following passage together with their properties.
                                            
                                            Passage:
                                            {input}
                                          """)

In [225]:
entity_schema = {
      "properties": {
          "person-name": "string", 
          "person-height": "integer",
          "person-hair_color": "string",
          "dog-name": "string",
          "dog-breed": "string"
              },
    "required" : []
    }

In [226]:
inp = """
Alex is 5 feet long. Claudia is 4 feet than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
        """

In [227]:
chain = create_tagging_chain(entity_schema, llm, prompt)

chain({'input':inp})

{'input': "\nAlex is 5 feet long. Claudia is 4 feet than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n        ",
 'output': AIMessage(content='', additional_kwargs={'function_call': {'name': 'entity_tagging', 'arguments': '{\n  "person-name": ["Alex", "Claudia"],\n  "person-height": ["5 feet", "4 feet"],\n  "person-hair_color": ["blonde", "brunette"],\n  "dog-name": ["Frosty"],\n  "dog-breed": ["labrador"]\n}'}}, example=False)}

In [217]:
prompt = ChatPromptTemplate.from_template("""
                                            Extract the desired information from the following passage.
                                            
                                            Passage:
                                            {input}
                                          """)

In [211]:
entity_schema = {
      "properties": {
          "sentiment": "string", 
          "tone": "string",
          "language": "string",
              },
    "required": ['tone']
    }

In [213]:
inp = """
Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos! 
        """

In [214]:
chain = create_extraction_chain(entity_schema, llm, prompt)

chain({'input':inp})

{'input': '\nEstoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos! \n        ',
 'output': AIMessage(content='', additional_kwargs={'function_call': {'name': 'information_extraction', 'arguments': '{\n  "tone": "positive",\n  "language": "Spanish"\n}'}}, example=False)}